# 子词嵌入Subword Embedding

## 1. fastText模型

$$ \mathbf{v}_w = \sum_{g\in\mathcal{G}_w} \mathbf{z}_g $$

## 2. 字节对编码（Byte Pair Encoding）

In [ ]:
import collections

# 预定义所有可能的符号，包括小写字母、下划线和未知标记
symbols = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
           'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
           '_', '[UNK]']

In [ ]:
# 定义原始token及其频率
raw_token_freqs = {'fast_': 4, 'faster_': 3, 'tall_': 5, 'taller_': 4}
token_freqs = {}
# 把每个单词按字符分开，用空格连接（为BPE合并做准备）
for token, freq in raw_token_freqs.items():
    token_freqs[' '.join(list(token))] = raw_token_freqs[token]
token_freqs

In [ ]:
def get_max_freq_pair(token_freqs):
    """
    统计所有token中出现频率最高的相邻符号对
    """
    pairs = collections.defaultdict(int)
    for token, freq in token_freqs.items():
        symbols = token.split()
        for i in range(len(symbols) - 1):
            # 统计每个连续符号对出现的总频率
            pairs[symbols[i], symbols[i + 1]] += freq
    # 返回频率最高的符号对
    return max(pairs, key=pairs.get)  # 具有最大值的“pairs”键

In [ ]:
def merge_symbols(max_freq_pair, token_freqs, symbols):
    """
    把频率最高的符号对合并成新符号，并更新所有token
    """
    # 新增合并的符号
    symbols.append(''.join(max_freq_pair))
    new_token_freqs = dict()
    for token, freq in token_freqs.items():
        # 把token里的该符号对替换为合并后的新符号
        new_token = token.replace(' '.join(max_freq_pair),
                                  ''.join(max_freq_pair))
        new_token_freqs[new_token] = token_freqs[token]
    return new_token_freqs

In [ ]:
num_merges = 10 # 设置合并次数
for i in range(num_merges):
     # 找到最常见的符号对
    max_freq_pair = get_max_freq_pair(token_freqs)
     # 合并
    token_freqs = merge_symbols(max_freq_pair, token_freqs, symbols)
    print(f'合并# {i+1}:',max_freq_pair) # 打印每次合并的结果

In [ ]:
def segment_BPE(tokens, symbols):
    """
    用BPE合并得到的符号表对新单词分词
    """
    outputs = []
    for token in tokens:
        start, end = 0, len(token)
        cur_output = []
        # 尝试用符号表中最长的子串分割token
        while start < len(token) and start < end:
            if token[start: end] in symbols:
                cur_output.append(token[start: end])
                start = end
                end = len(token)
            else:
                end -= 1
        # 如果有部分无法匹配，标记为[UNK]
        if start < len(token):
            cur_output.append('[UNK]')
        outputs.append(' '.join(cur_output))
    return outputs

In [ ]:
# 新词
tokens = ['tallest_', 'fatter_']
# 用BPE分词并输出结果
print(segment_BPE(tokens, symbols))